In [2]:
!pip install tokenizers

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


# Load unstructured Datset

In [4]:
import datasets

In [5]:
# check list of all datasets
all_ds = datasets.list_datasets()
len(all_ds)

<ipython-input-5-85d703febd7a>:2: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  all_ds = datasets.list_datasets()


86957

In [6]:
all_ds[:5]

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus']

In [7]:
# one of these is oscar
'oscar' in all_ds

True

In [8]:
# dataset variable stores unshuffled_deduplicated_hi (hindi) of oscar dataset
dataset = datasets.load_dataset('oscar', 'unshuffled_deduplicated_hi')

Generating train split:   0%|          | 0/1909387 [00:00<?, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 1909387
    })
})

In [10]:
dataset['train'][3]

{'id': 3,
 'text': 'बेंगलुरु के आसमान में इंडिगो के 2 विमान टकराने से बचे, सिर्फ 200 फीट दूर रहते टला हादसा – Dot A Blog\nबेंगलुरु के आसमान में इंडिगो एयरलाइंस के दो विमान टकराने से बचे। दोनों के बीच एक-दूसरे से ऊंचाई में सिर्फ 200 फीट का फासला रह गया था, लेकिन सूचना मिलते ही पायलट ने विमानों के मार्ग बदल दिए। दोनों में 328 यात्री सवार थे। घटना मंगलवार की है। एयरलाइंस की ओर से जानकारी गुरुवार को दी गई।\nPrevious Postथाईलैंड के बचाव अभियान पर हॉलीवुड में 400 करोड़ रुपए में फिल्म बनेगी, गुफा को म्यूजियम में बदला जा सकता है\nNext Postक्रोएशिया पहली बार फुटबॉल विश्व कप के फाइनल में, 1 गोल से पिछड़ने के बाद एक्सट्रा टाइम में 2-1 से जीता मैच\nदेश में वोटिंग के लिए एनआरआई अब दूसरों को दे सकेंगे अधिकार, लोकसभा में जनप्रतिनिधित्व संशोधन बिल पास'}

In [11]:
df = dataset['train'].select(list(range(10000)))
df

Dataset({
    features: ['id', 'text'],
    num_rows: 10000
})

In [12]:
# we use tqdm to see whats happening: (progress bar,)etc
from tqdm.auto import tqdm

# initialize a list that contains text data
text_data = []
# Tokenizer needs every sample to be on different line. This is make proper file for tokenizer
file_count = 0

# We want to save our data
# this will show the loading/progress bar
for sample in tqdm(df):
  sample = sample['text'].replace('\n',' ')
  # add one sample to text datas list
  text_data.append(sample)

  # if the length of text_data list hits 1k, we want to create a file
  # Every file contains 1000 samples joined by new line
  if len(text_data) == 500:
    with open(f'it_{file_count}.txt', 'w',encoding = 'utf-8') as fp:
      fp.write('\n'.join(text_data))
    # we dont need previous text_data now so we want to reintialize
    text_data = []
    file_count += 1
  # this will save the remaining data (last batch)
  with open(f'it_{file_count}.txt', 'w',encoding = 'utf-8') as fp:
    fp.write('\n'.join(text_data))





  0%|          | 0/10000 [00:00<?, ?it/s]

 We have saved text files from the hindi subset from oscar dataset to our local system. Each sample is seperated by new line character

# Training the tokenizer

In [13]:
# path to all the files
from pathlib import Path

In [14]:
# this picks names all .txt filesas str in the dorectory
paths = [str(x) for x in Path('./').glob('*.txt')]
paths[:10]

['it_12.txt',
 'it_1.txt',
 'it_2.txt',
 'it_18.txt',
 'it_4.txt',
 'it_8.txt',
 'it_9.txt',
 'it_14.txt',
 'it_10.txt',
 'it_20.txt']

Tokenization can be done using different methods, such as `rule-based, statistical, or neural`, depending on the complexity and the variability of the texts. OpenAI and Azure OpenAI uses a subword tokenization method called `"Byte-Pair Encoding (BPE)"` for its GPT-based models

https://deepgram.com/ai-glossary/tokenization

In [15]:
# We are using byte level- byte pair tokenizer : it is breaking sentences to bytes
from tokenizers import  ByteLevelBPETokenizer

In [16]:
tokenizer = ByteLevelBPETokenizer()

In [17]:
tokenizer.train(files = paths[:10],vocab_size = 30000, min_frequency=2,
                special_tokens = [
                    '<s>','<pad>', '</s>', '<unk>', '<mask>'
                    ])

In [18]:
import os
os.mkdir('hindi_token')

In [19]:
tokenizer.save_model('hindi_token')

['hindi_token/vocab.json', 'hindi_token/merges.txt']

This vocab.json and merges.txt contains how tokens are represented and given a token id. This completes the tokenizer

* Loading the Trained Tokenizer using RobertaTokenizerFast (testing purpose)

In [20]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
   'hindi_token/vocab.json',
   'hindi_token/merges.txt'
)

In [21]:
text = 'आप कैसे हो'
tokens = tokenizer.encode(text).tokens
print(tokens)
encoded_text = tokenizer.encode(text)
encoded_text

['à¤Ĩà¤ª', 'Ġà¤ķ', 'à¥Ī', 'à¤¸', 'à¥ĩ', 'Ġà¤¹', 'à¥ĭ']


Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [22]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('hindi_token')

In [23]:
tokenizer('आप कैसे हो')

{'input_ids': [0, 1784, 268, 285, 278, 265, 279, 273, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenizer('आप कैसे हो', padding = 'max_length', max_length = 50)

{'input_ids': [0, 1784, 268, 285, 278, 265, 279, 273, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

# Input pipeline and Training

* Objectives :MLM (Masked Language Modeling) :
MLM is a language model trained to predict the missing words in a sentence based on the context provided by the surrounding words.
* Requiements ▶ `input_id`, `attention_masj`, `labels tensor`


The original input_ids are stored as labels
The input ids are changed and will be assigned from a MLM script that will mask the 15% text of the tokens within the tensor.
input_ids -> MLM
During training the model tries to detect these masked tokens.

In [25]:
import torch

In [26]:
# create a mask language modelling function
def mlm(tensor):

  # create a random array with same shape as our input tensor
  rand = torch.rand(tensor.shape) #[0,1] every value is between 0 and 1

  # now we want to select around 15% of the tensor values : and skip the special tokens like(0,1,2- token values)
  # mask_arr = (rand < 0.15)*(tensor != 0)*(tensor != 1)*(tensor != 2)
  mask_arr = (rand < 0.15)*(tensor > 2 ) # gives true if random value is less than 0.15

  # loop through each row in tensor( and this cannot be done in parallel as each row will have differernt number of values)
  # torch.flatten is a function that is used to flatten a multi-dimensional tensor into a one-dimensional tensor
  for i in range(tensor.shape[0]):

    # mask_arr[i].nonzero() gives the indices of non-zero elements in the i-th row of mask_arr
    selection = torch.flatten(mask_arr[i].nonzero()) # eg [[2,4,18]]-(tolist)-> [2,4,18], etc

    # change ith row and selection (cloumn) to <mask> token(as present in vocab.json)
    tensor[i,selection] = 4
  return tensor




In [27]:
max_length = 10

In [28]:
input_ids = []
# attentio mask
masks = []
# labels
labels = []
for path in tqdm(paths[:10]):
  with open(path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n') # list of samples
  # Tokenization
  samples = tokenizer(lines, max_length = max_length, padding='max_length', truncation= True)
  input_ids_tensor = torch.tensor(samples['input_ids'])
  attention_mask_tensor = torch.tensor(samples['attention_mask'])
  labels.append(input_ids_tensor)
  masks.append(attention_mask_tensor)
  input_ids.append(mlm(input_ids_tensor)) #clone the input_id array


  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
input_ids = torch.cat(input_ids) # list of tensors
masks = torch.cat(masks)
labels = torch.cat(labels)

In [30]:
input_ids[0] # 4 -> masked

tensor([   0, 3391,  267,  274,  265,  298,  273,  278,  267,    2])

In [31]:
# now we change the format to dictionary
encodings = {
    'input_ids' : input_ids,  # appx 15% masked
    'attention_mask' : masks, # attentions
    'labels' : labels # real tokens
}

In [32]:
encodings.items()
encodings['input_ids']

tensor([[   0, 3391,  267,  ...,  278,  267,    2],
        [   0,  304,  353,  ...,  309,  268,    2],
        [   0,  288,  267,  ...,  274,  267,    2],
        ...,
        [   0,  283,  265,  ...,  274,  264,    2],
        [   0,  289,  265,  ...,  264,  275,    2],
        [   0,    2,    1,  ...,    1,    1,    1]])

In [33]:
# dataset object which is used to load data : input pipeline
# Create a torch dataset
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings):
    self.encodings = encodings
  def __len__(self):
    return self.encodings['input_ids'].shape[0]
  def __getitem__(self,i):
    return{ key : tensor[i] for key,tensor in self.encodings.items() }

In [34]:
# we have specified datasets object
dataset = Dataset(encodings)

In [35]:
# use torch dataset to create torch dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle= True)

# Training our Model

Before training we need to set up certain configuration
* `Roberta Config Object` : We want to set certain set of parameters
* `RobertaForMaskedLM` : or 'RobertaForCasualLM`ie select a plain model and send the desired configuration to the model
* `Optimezier and learning rate` : set both
* use `data loader` to load the data and train the model

In [36]:
from transformers import RobertaConfig

In [37]:
tokenizer.vocab_size

21306

In [38]:
config = RobertaConfig(
    vocab_size = tokenizer.vocab_size,  # this needs to match with vocab_size which we have used for tokenizer
    max_position_embeddings = max_length + 2,
    hidden_size = 768, # value from bert base model
    num_attention_heads = 12,
    num_hidden_layers = 6, # or 12(as for roberta),
    type_vocab_size = 1
)

In [39]:
from transformers import RobertaForMaskedLM #: for mlm
# from transformers import RobertaForCausalLM : for cmm


## model initialization

In [40]:
# plain roberta model initialized
model = RobertaForMaskedLM(config)

In [41]:
# Decide the device that will be used to train the model
torch.cuda.is_available()

False

In [42]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [43]:
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(21306, 768, padding_idx=1)
      (position_embeddings): Embedding(12, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

## optimizer

In [44]:
from transformers import AdamW

In [45]:
optim = AdamW(model.parameters(), lr = 5e-1)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [46]:
epoch = 2

In [47]:
loop = tqdm(dataloader, leave = True)
for batch in loop:
  # initialize gradient to zero at start of every loop
  optim.zero_grad()
  input_ids = batch['input_ids'].to(device)
  mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  output = model(input_ids, attention_mask = mask, labels = labels)
  loss = output.loss
  loss.backward() # back propogate
  optim.step() # optimize
  loop.set_description(f'Epoch: {epoch}')
  loop.set_postfix(loss = loss.item())

  0%|          | 0/282 [00:00<?, ?it/s]

# Testing

In [48]:
from transformers import pipeline

In [52]:
fill = pipeline('fill-mask', model = model, tokenizer = tokenizer)

In [53]:
fill(f'आप {fill.tokenizer.mask_token} रहे हैं ?')

[{'score': 1.0, 'token': 0, 'token_str': '<s>', 'sequence': 'आप रहे हैं?'},
 {'score': 2.8878645440606066e-28,
  'token': 328,
  'token_str': ' च',
  'sequence': 'आप च रहे हैं?'},
 {'score': 8.179359689380181e-34,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'आप रहे हैं?'},
 {'score': 0.0, 'token': 1, 'token_str': '<pad>', 'sequence': 'आप रहे हैं?'},
 {'score': 0.0, 'token': 3, 'token_str': '<unk>', 'sequence': 'आप रहे हैं?'}]